This is a notebook to pull down all the information for figure 3 (ncbi hits). The design is to compair between each sample (barcode) instead of each flowcell as they sequenced the same thing. Therefore I just pulled all the ncbi hits for each flowcell for each barcode and merged them together according to the barcode.

In [2]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from itertools import product
import argparse
from ete3 import NCBITaxa 

In [3]:
ncbi = NCBITaxa()

In [4]:
# use argparse to do this
#BASEDIR = args.BASEDIR
BASEDIR = '/home/yiheng/data/20170617_FAH05432'

# here we define the folder name of the dataframe it created by capturing the folder from the BASDIR
folder_name = os.path.basename(BASEDIR)
column_name = folder_name.split('_')[-1]

In [5]:
# first check if the analysis folder is there
folder_list = 'analysis  basecalled_data  scripts  tracking  workspace'.split(' ')
for x in range(0,folder_list.count('')):
    folder_list.remove('')
#fix this test
if not set(os.listdir(os.path.abspath(BASEDIR))) == set (folder_list):
    print("Something wrong with basefolder. check it please.")

In [6]:
# get the dataframe there
dataframe = os.path.join(BASEDIR, 'analysis', 'summary_df_%s.tab' % folder_name)
sum_df = pd.read_csv(dataframe, sep='\t')

/home/yiheng/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,2,3,5,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
sum_df.columns

Index(['read_id', 'passes_filtering', 'sequence_length_template',
       'mean_qscore_template', 'barcode_arrangement', 'barcode_score', 'kit',
       'variant', 'pc_survived', 'nl_survived', 'qseqid_rg', 'sseqid_rg',
       'evalue_rg', 'length_rg', 'pident_rg', 'nident_rg', 'sacc_rg',
       'staxids_rg', 'scomnames_rg', 'read_length_pc_x', 'qseqid_nt',
       'sseqid_nt', 'evalue_nt', 'length_nt', 'pident_nt', 'nident_nt',
       'sacc_nt', 'staxids_nt', 'scomnames_nt', 'read_length_pc_y'],
      dtype='object')

In [8]:
# fills the nan with T/F for easier handling
sum_df.sseqid_rg.fillna(False, inplace=True)
sum_df.sseqid_nt.fillna(False, inplace=True)
# filter out the rg blast hit
ntblasthit_reads = sum_df[(sum_df.sseqid_rg == False) & (sum_df.sseqid_nt != False) & (sum_df.passes_filtering == True) & (sum_df.pc_survived == True) & (sum_df.nl_survived == True)]
pd.set_option('display.max_columns', None)

#Removing unncessary columns from joint_NCBI_df_taxonomy
#I hard coded to select as it depends on what information you need
for columns in ['passes_filtering', 'sequence_length_template',
       'mean_qscore_template', 'barcode_score', 'kit',
       'variant', 'pc_survived', 'nl_survived', 'qseqid_rg', 'sseqid_rg',
       'evalue_rg', 'length_rg', 'pident_rg', 'nident_rg', 'sacc_rg',
       'staxids_rg', 'scomnames_rg', 'read_length_pc_x', 'qseqid_nt',
       'sseqid_nt', 'evalue_nt', 'length_nt', 'pident_nt', 'nident_nt',
       'sacc_nt','scomnames_nt', 'read_length_pc_y']:
    del ntblasthit_reads[columns] 

In [9]:
###########This block for barcodes order
# now everything left is what we want to plot out
# arrange different barcodes
# REMEMBER: This need to be manual check:
# barcode01 is Pst infected sample, barcode02 is Zymo sample, 03 is nill sample, 04 is co-infection sample and 05 is Pyre sample
new_barcodes = ['barcode01', 'barcode02', 'barcode03', 'barcode04', 'barcode05']
if column_name == 'run1':
    ntblasthit_barcode01 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode02']
    ntblasthit_barcode02 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode06']
    ntblasthit_barcode03 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode04']
    ntblasthit_barcode04 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode05']
    ntblasthit_barcode05 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode03']
    ntblasthit_barcode01.barcode_arrangement = ntblasthit_barcode01.barcode_arrangement.replace(to_replace='barcode02', value='barcode01')
    ntblasthit_reads.drop(ntblasthit_reads.index[ntblasthit_reads.barcode_arrangement.str.contains('barcode07')], inplace = True)
    ntblasthit_barcode01.barcode_arrangement = ntblasthit_barcode01.barcode_arrangement.replace(to_replace='barcode03', value='barcode07')
    ntblasthit_barcode01.barcode_arrangement = ntblasthit_barcode01.barcode_arrangement.replace(to_replace='barcode06', value='barcode02')
    ntblasthit_barcode01.barcode_arrangement = ntblasthit_barcode01.barcode_arrangement.replace(to_replace='barcode04', value='barcode03')
    ntblasthit_barcode01.barcode_arrangement = ntblasthit_barcode01.barcode_arrangement.replace(to_replace='barcode05', value='barcode04')
    ntblasthit_barcode01.barcode_arrangement = ntblasthit_barcode01.barcode_arrangement.replace(to_replace='barcode07', value='barcode05')

elif column_name == 'run2':
    # first delete the rows that contains the barcode we are going to replace
    # I think maybe better to first adjust the df to get rid of all the mis/unclassified reads so the df get smaller
    # and we do not need those reads for plotting
    
    ntblasthit_reads.drop(ntblasthit_reads.index[ntblasthit_reads.barcode_arrangement.str.contains('barcode01')], inplace = True)
    ntblasthit_reads.drop(ntblasthit_reads.index[ntblasthit_reads.barcode_arrangement.str.contains('barcode02')], inplace = True)
    ntblasthit_reads.drop(ntblasthit_reads.index[ntblasthit_reads.barcode_arrangement.str.contains('barcode03')], inplace = True)
    ntblasthit_reads.drop(ntblasthit_reads.index[ntblasthit_reads.barcode_arrangement.str.contains('barcode04')], inplace = True)
    ntblasthit_reads.drop(ntblasthit_reads.index[ntblasthit_reads.barcode_arrangement.str.contains('barcode05')], inplace = True)
    ntblasthit_reads.drop(ntblasthit_reads.index[ntblasthit_reads.barcode_arrangement.str.contains('unclassified')], inplace = True)
    old_barcodes = ['barcode07', 'barcode08', 'barcode09', 'barcode10', 'barcode11']
    for old_barcode, new_barcode in zip(old_barcodes, new_barcodes):
        ntblasthit_reads.loc\
        [ntblasthit_reads[ntblasthit_reads['barcode_arrangement'] == old_barcode].index,'barcode_arrangement']\
        = [new_barcode]
    #ntblasthit_reads['barcode_arrangement'] = ntblasthit_reads.barcode_arrangement.replace(to_replace='barcode07', value='barcode01')

    
elif column_name == 'FAH05432':
    ntblasthit_barcode01 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode01']
    ntblasthit_barcode02 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode02']
    ntblasthit_barcode03 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode03']
    ntblasthit_barcode04 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode04']
    ntblasthit_barcode05 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode05']
elif column_name == 'FAH05731':
    ntblasthit_barcode01 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode01']
    ntblasthit_barcode02 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode02']
    ntblasthit_barcode03 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode03']
    ntblasthit_barcode04 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode04']
    ntblasthit_barcode05 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement == 'barcode05']
else:
    pass



In [10]:
# now just filter out all the reads that are not going to used for ploting
ntblasthit_reads_barcode01 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement.str.contains('barcode01')]
ntblasthit_reads_barcode02 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement.str.contains('barcode02')]
ntblasthit_reads_barcode03 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement.str.contains('barcode03')]
ntblasthit_reads_barcode04 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement.str.contains('barcode04')]
ntblasthit_reads_barcode05 = ntblasthit_reads[ntblasthit_reads.barcode_arrangement.str.contains('barcode05')]

# now concat them together. but remember that the index has to change to add the column for ncbi taxa
ntblasthit_reads_filtered_barcodes = pd.concat([ntblasthit_reads_barcode01, 
                                                ntblasthit_reads_barcode02, 
                                                ntblasthit_reads_barcode03, 
                                                ntblasthit_reads_barcode04, 
                                                ntblasthit_reads_barcode05], ignore_index=True)

In [11]:
# as the concate before will result the staxid into a string
# so now change the string back to float so it can be recognized by the NCBITaxa
for taxid in ntblasthit_reads_filtered_barcodes['staxids_nt']:
    if ';' in str(taxid):
        taxid = taxid.split(';')[0]
    else:
        pass
    float(taxid)

In [12]:
#Function to generate taxonomy columns based on NCBITaxa results for NCBI hit dataframe, 
def search_rank_output_name_append_column(df, staxid_column, rank_search):
    """Input df, staxid_column from same df and rank_search (a desired taxonomic rank 
    from each staxid's lineage), outputs taxonomic name corresponding to rank_search or 'Unclassified' if
    unavailable and appends to df row by row"""
    rank_list = []
    for read_index in range(0, len(staxid_column)):
        taxid = ''
        if ';' in str(staxid_column[read_index]):
            taxid = staxid_column[read_index].split(';')[0]
        else:
            taxid = staxid_column[read_index]
        
        taxid_lineage = ''
        taxid_lineage = ncbi.get_lineage(taxid)
        
        names = ''
        names = ncbi.get_taxid_translator(taxid_lineage)
        
        ranks = ''
        ranks = ncbi.get_rank(taxid_lineage) #Dict
        
        ranks2names = ''
        ranks2names = {ranks[k]:names[k] for k in names.keys() & ranks}
        
        if rank_search in ranks2names.keys():
            rank_list.append(ranks2names[rank_search])#if rank in dict, print name
        else:
            rank_list.append('Unclassified')
    df[rank_search] = rank_list
# NOTE: Appending is always slow, try and find a better way e.g df.apply to a column based on staxids column


In [13]:
#ntblasthit_reads_filtered_barcodes_added_TaxaRank = ntblasthit_reads_filtered_barcodes.copy()
rank_list = ['superkingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
for rank in rank_list:
    search_rank_output_name_append_column(ntblasthit_reads_filtered_barcodes, 
                                          ntblasthit_reads_filtered_barcodes.staxids_nt, 
                                          rank)

In [14]:
ntblasthit_reads_filtered_barcodes

,read_id,barcode_arrangement,staxids_nt,superkingdom,phylum,class,order,family,genus,species
0,807a8997-b021-4820-b33a-1b9095249dc7,barcode01,200451,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Pseudomonas poae
1,c53fa9c8-f4d5-4876-8526-e04fcea0ac5d,barcode01,200451,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Pseudomonas poae
2,19a62259-0925-41ef-94e4-6fe9ef776c81,barcode01,199201,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Pseudomonas syringae
3,7e4e83ba-26fa-4d0b-b714-724976a9c7a7,barcode01,200451,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Pseudomonas poae
4,c92e3d57-c554-4d73-9e17-a51bc14aa4d4,barcode01,1.35728e+06,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Pseudomonas syringae
5,072bbc16-20a5-45b8-bcbf-e7239f48ed12,barcode01,1.32493e+06,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Pseudomonas syringae
6,7eb68975-bd44-40da-b0e8-2635fcedfbb0,barcode01,716540,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Erwiniaceae,Erwinia,Erwinia amylovora
7,f876c268-fe76-4fb7-97e7-e3c6fa2d8c8b,barcode01,1.28236e+06,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Pseudomonas poae
8,00f1b4d2-cc23-4231-9b51-543873ddddd2,barcode01,634500,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Erwiniaceae,Erwinia,Erwinia billingiae
9,74885d13-7285-4de6-ae85-89a8927c36ab,barcode01,1.50567e+06,Eukaryota,Basidiomycota,Pucciniomycetes,Pucciniales,Pucciniaceae,Puccinia,Puccinia cf. psidii AE-2014


In [14]:
# add the flowcell column just in case
ntblasthit_reads_filtered_barcodes = ntblasthit_reads_filtered_barcodes.assign(Flowcell = column_name)

In [15]:
ntblasthit_reads_filtered_barcodes.to_csv(r'/home/yiheng/analysis/WGS/%s_taxa.tab' % column_name, header=column_name, index=None, sep='\t')

In [16]:
#Defines a function to make manual adjustments to the NCBI taxonomic sorting
def manually_adjust_ncbi_taxonomy(rank_select, entry_select, ranks_to_edit, new_entry):
    """Select dataframe entries to edit by a specific rank (rank_select) 
    and entry related to that rank (entry_select), 
    provide a list of ranks to edit for those selected entries (ranks_to_edit),
    replace these ranks with a new entry (new_entry)"""
    if entry_select in list(ntblasthit_reads_filtered_barcodes[rank_select].unique()):
        selection = (ntblasthit_reads_filtered_barcodes[rank_select] == entry_select)
        selected_tax_ranks = ranks_to_edit
        for ranks in selected_tax_ranks:
            ntblasthit_reads_filtered_barcodes[ranks][selection] = new_entry

In [17]:
manually_adjust_ncbi_taxonomy('species', 
                            'Cloning vector lambda TXF97', 
                            ['superkingdom', 'phylum', 'class', 'order', 'family', 'genus'], 
                            'Cloning vector lambda TXF97')

manually_adjust_ncbi_taxonomy('superkingdom', 
                            'Viruses', 
                            ['phylum', 'class'], 
                            'dsDNA viruses, no RNA stage')

In [18]:
#count pivot table of dataframe with taxonomic columns: 
#rows - rank names, columns - barcode, sorted by desired class
def generate_ncbi_taxonomy_pivot(tax_df, rank, bcs, num):
    """From tax_df, generate a pivot table listing num rank counts, sorted by bcs"""
    pivot_table = tax_df.pivot_table(values='order', 
                                            index=rank, 
                                            columns='barcode_arrangement', 
                                            aggfunc='count', 
                                            fill_value=0, 
                                            margins=True)
    pivot_table.columns.name = None
    return pivot_table.sort_values(bcs, axis=0, ascending=False).head(n=num)

In [19]:
generate_ncbi_taxonomy_pivot(ntblasthit_reads_filtered_barcodes, 'family','barcode04',20)

,barcode01,barcode02,barcode03,barcode04,barcode05,All
family,,,,,,
All,1942,1855,37,1748,271,5853
Pseudomonadaceae,1470,814,0,1104,58,3446
Pleosporaceae,44,263,1,160,77,545
Sphingomonadaceae,1,161,0,83,1,246
Mycosphaerellaceae,1,103,0,43,2,149
Hymenobacteraceae,1,82,0,39,0,122
Enterobacteriaceae,51,56,10,35,5,157
Microbacteriaceae,3,37,0,31,0,71
Poaceae,14,8,13,29,41,105


In [20]:
# % pivot table of dataframe with taxonomic columns: 
#rows - rank names, columns - barcode, sorted by desired class
def generate_ncbi_taxonomy_pivot_pc(tax_df, rank, bcs, num):
    """From tax_df, generate a pivot table listing num rank %, sorted ascending by bcs"""
    pivot_table_pc = tax_df.pivot_table(values='order', 
                                            index=rank, 
                                            columns='barcode_arrangement', 
                                            aggfunc='count', 
                                            fill_value=0, 
                                            margins=True)
    pivot_table_pc.columns.name = None
    
    for brcs in pivot_table_pc.columns:
        pivot_table_pc[str(brcs) + '%'] = pivot_table_pc[brcs].apply(lambda x: (x / pivot_table_pc[brcs].loc['All'])*100)
    for brcs in pivot_table_pc.columns:
        if '%' not in brcs:
            pivot_table_pc.drop(brcs, axis=1, inplace=True)
    pivot_table_pc.drop('All', axis=0, inplace=True)

    pivot_table_pc2 = pivot_table_pc.sort_values(str(bcs) + '%', axis=0, ascending=False)
    return pivot_table_pc2.applymap(lambda x: '%.3f' % x).head(n=num)

In [21]:
generate_ncbi_taxonomy_pivot_pc(ntblasthit_reads_filtered_barcodes, 'genus', 'barcode04', 20)

,barcode01%,barcode02%,barcode03%,barcode04%,barcode05%,All%
genus,,,,,,
Pseudomonas,75.695,43.881,0.000,63.158,21.402,58.876
Alternaria,1.442,11.321,2.703,7.037,20.295,7.125
Sphingomonas,0.051,7.925,0.000,4.291,0.369,3.827
Zymoseptoria,0.051,5.337,0.000,2.346,0.000,2.409
Hymenobacter,0.051,4.420,0.000,2.231,0.000,2.084
Escherichia,0.566,2.426,24.324,1.716,1.476,1.691
Bipolaris,0.412,1.456,0.000,1.201,4.059,1.145
Curtobacterium,0.051,0.755,0.000,1.144,0.000,0.598
Leptosphaeria,0.360,1.833,2.703,1.030,6.273,1.316


In [22]:
#Assesses number of barcodes, generates necessary number of plots (even) and relevant indexing system
ncbiblast_barcodes = list(ntblasthit_reads_filtered_barcodes['barcode_arrangement'].unique())
no_of_subplots = len(ncbiblast_barcodes)+1

In [23]:
#Always 2 columns, bc-count/2 rows 
no_of_subplots_pair = [int(no_of_subplots/2), 2]

#Produce pairs of indices correlating to the coordinates of the subplots
subplot_coordinates = list(product(range(no_of_subplots_pair[0]), range(no_of_subplots_pair[1])))
subplot_coordinates_list = [list(l) for l in subplot_coordinates]
subplot_coordinates_list_rows = [i[0] for i in subplot_coordinates_list]
subplot_coordinates_list_columns = [i[1] for i in subplot_coordinates_list]

fig, ax = plt.subplots(no_of_subplots_pair[1], no_of_subplots_pair[0], figsize=(20, 6))

In [24]:
name_list = ['Stripe rust', 'Septoria tritici blotch', 'Healthy wheat', 'Stripe rust and sptoria tritici blotch', 'Yellow spot']

In [ ]:
def get_color(x):
    x = x%10
    return ('#%02X%02X%02X' % (int(plt.cm.tab10.colors[x][0]*255),int(plt.cm.tab10.colors[x][1]*255),int(plt.cm.tab10.colors[x][2]*255)))

def applyTaxPlotStyle(ax_ind1, ax_ind2, bcs, rank, name):
    labels = ''
    labels = tuple(generate_ncbi_taxonomy_pivot_pc(ntblasthit_reads_filtered_barcodes, rank, bcs, 10)[bcs + '%'].index)
    y_pos = ''
    y_pos = np.arange(len(labels))
    sizes = ''
    sizes = tuple([float(i) for i in generate_ncbi_taxonomy_pivot_pc(ntblasthit_reads_filtered_barcodes, rank, bcs, 10)[bcs + '%']])
    
    ax[ax_ind1, ax_ind2].barh(y_pos, sizes, align='center', alpha=0.4, color=get_color(2*ax_ind2 + ax_ind1))
    ax[ax_ind1, ax_ind2].set_yticks(y_pos)
    ax[ax_ind1, ax_ind2].set_yticklabels(labels)
    ax[ax_ind1, ax_ind2].tick_params(top="off", right="off", left="off")
    ax[ax_ind1, ax_ind2].set_xlabel('% Total Reads Hitting NCBI')
    ax[ax_ind1, ax_ind2].set_xticks(range(0, 110, 10))
    ax[ax_ind1, ax_ind2].set_title(name)
    ax[ax_ind1, ax_ind2].invert_yaxis()
    ax[ax_ind1, ax_ind2].grid(b=None, which='major', axis='x', linestyle='dashed', linewidth=0.2)
    ax[ax_ind1, ax_ind2].annotate('E = 0.00001', 
             xy=(1.06, 0.3), xycoords='axes fraction', fontsize=10, xytext=(-20, -30), textcoords='offset points', 
             ha='right', va='bottom')

    
def generateTaxSubplots(search_rank):
    for ax_ind1, ax_ind2, bcs, name in zip(subplot_coordinates_list_columns, subplot_coordinates_list_rows, ncbiblast_barcodes, name_list):
        applyTaxPlotStyle(ax_ind1, ax_ind2, bcs, search_rank, name)    

    if len(list(ntblasthit_reads_filtered_barcodes['barcode_arrangement'].unique())) != 0:
        plt.delaxes(ax[subplot_coordinates_list_columns[-1], subplot_coordinates_list_rows[-1]])
    
    plt.suptitle('Reads Hitting NCBI Database - % By ' + str(search_rank).title() + ' By Sample', 
             fontsize='x-large',
             y=1.02,
             fontweight="bold")
    
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=1.4, hspace=0.4)
    plt.tight_layout()
    
    plt.savefig('/home/yiheng/analysis/WGS/figures' + 'figure3_%s_%s.png' % (column_name, search_rank), bbox_inches='tight', dpi=1000)
    plt.show()
            
            


In [ ]:
fig, ax = plt.subplots(no_of_subplots_pair[1], no_of_subplots_pair[0], figsize=(20, 6))
generateTaxSubplots('genus')

In [ ]:
df1=pd.DataFrame([[1, 2],[10, 1], [3, 4], [7, 8]], columns=['A', 'B'])
df2=pd.DataFrame([[1, 4],[3, 5]], columns=['A', 'B'])
# now replace the location use the index of each df selected from value by .loc function
# first select the value from df2 column A
for values in df2['A'].unique():
    # then locate the position in df1 and replace it by the VALUE of the position in df2
    df1.loc[df1[df1['A'] == (values)].index, 'B'] = df2.loc[df2[df2['A'] == (values)].index, 'B'].values

In [ ]:
df1